# 환경변수 불러오기

In [21]:
from dotenv import load_dotenv 
load_dotenv()

True

# STT(Speech to Text)

In [22]:
import speech_recognition as sr
print(sr.Microphone.list_microphone_names())

['Microsoft 사운드 매퍼 - Input', '마이크 배열(디지털 마이크용 인텔® 스마트 사운드 기술)', 'CABLE Output(VB-Audio Virtual C', 'Microsoft 사운드 매퍼 - Output', 'LS27F61x(3- HD Audio Driver for', 'CABLE In 16ch(VB-Audio Virtual ', 'CABLE Input(VB-Audio Virtual Ca', '스피커(Realtek(R) Audio)', '주 사운드 캡처 드라이버', '마이크 배열(디지털 마이크용 인텔® 스마트 사운드 기술)', 'CABLE Output(VB-Audio Virtual Cable)', '주 사운드 드라이버', 'LS27F61x(3- HD Audio Driver for Display Audio)', 'CABLE In 16ch(VB-Audio Virtual Cable)', 'CABLE Input(VB-Audio Virtual Cable)', '스피커(Realtek(R) Audio)', 'LS27F61x(3- HD Audio Driver for Display Audio)', 'CABLE In 16ch(VB-Audio Virtual Cable)', 'CABLE Input(VB-Audio Virtual Cable)', '스피커(Realtek(R) Audio)', 'CABLE Output(VB-Audio Virtual Cable)', '마이크 배열(디지털 마이크용 인텔® 스마트 사운드 기술)', 'CABLE Output (VB-Audio Point)', 'Output (VB-Audio Point)', 'Input (VB-Audio Point)', '스테레오 믹스 (Realtek HD Audio Stereo input)', 'Headphones 1 (Realtek HD Audio 2nd output with SST)', 'Headphones 2 (Realtek HD Audio 2nd output with SST)', 'PC 스피커 (Rea

In [23]:
# uv add pyaudio speechrecognition pydub
import speech_recognition as sr

r = sr.Recognizer() 
# 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
with sr.Microphone(device_index=1) as source:
    print("말해주세요:")
    r.adjust_for_ambient_noise(source,duration=1) # 주변 소음 조정
    audio = r.listen(source)         # STEP1: 마이크 입력 받기
    print("인식 중입니다.....")
    text = r.recognize_openai(audio) # STEP2: 텍스트 변환
    print(f"인식된 텍스트: {text}")

    # STEP3: 마이크에 입력된 오디오 녹음하기
    audio_file = audio.get_wav_data()
    with open("./data/input.wav", "wb") as f:
        f.write(audio_file)
    print("목소리 저장 완료!")

말해주세요:


KeyboardInterrupt: 

In [24]:
# 오디오 출력하기 
from pydub import AudioSegment 
from pydub.playback import play

sound = AudioSegment.from_wav("./data/input.wav")
play(sound)

# LLM 연결하기

In [25]:
from openai import OpenAI 

client = OpenAI()

def chat(user_text):
    system_prompt = """당신은 시니컬한 챗봇입니다. 항상 50자 이내로 말해주세요"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_text}
        ]
    )

    return response.choices[0].message.content

# 챗봇과 대화해보기

In [ ]:
while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("말해주세요:")
        r.adjust_for_ambient_noise(source) # 주변 소음 조정
        audio = r.listen(source)         # STEP1: 마이크 입력 받기
        print("인식 중입니다.....")
        user_text = r.recognize_openai(audio) # STEP2: 텍스트 변환
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break

        answer = chat(user_text)
        print(f"챗봇 답변: {answer}")

# TTS(Text to Speech)

In [44]:
from openai import OpenAI 

client = OpenAI()

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="안녕하세요. 반가워요!!!!!",
    instructions="너는 미국 사람인데 서툴은 한국말이며 나이는 29살이고 특이한 목소리를 가진 사람입니다. 엄청 화난 목소리야 세상에서 무엇보다"
) as response:
    response.stream_to_file("./data/speech.mp3")

In [45]:
from pydub import AudioSegment 
from pydub.playback import play 

sound = AudioSegment.from_mp3("./data/speech.mp3")
play(sound)

# 인공지능스피커 만들기

In [ ]:
import os
import tempfile

while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("📢 듣는 중....")
        # STEP 1. 마이크로부터 입력
        r.adjust_for_ambient_noise(source) 
        audio = r.listen(source)         
        print("인식 중입니다.....")

        # STEP 2. Whisper API를 통한 텍스트 변환
        user_text = r.recognize_openai(audio) 
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break
        
        # STEP 3. 인공지능 챗봇 응답
        answer = chat(user_text)
        print(f"챗봇 답변: {answer}")

        # STEP 4. Whisper API로 응답
        with client.audio.speech.with_streaming_response.create(
            model="gpt-4o-mini-tts",
            voice="coral",
            input=answer,
            instructions="밝은 목소리로 말해줘"
        ) as response:
            # 음성 합성 결과를 임시 파일로 저장
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
                temp_path = temp_file.name
                response.stream_to_file(temp_path)

                # 재생
                sound = AudioSegment.from_mp3(temp_path)
                play(sound)

듣는 중....
인식 중입니다.....
인식된 텍스트: You
챗봇 답변: 뭐, 필요한 거라도 있나?


PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpejh7c9kq.mp3'